In [2]:
import os

import autogen
from autogen.coding import LocalCommandLineCodeExecutor
from autogen import ConversableAgent
from dotenv import load_dotenv

load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")
nvidia_api_key = os.getenv("NVIDIA_API_KEY")
open_api_key = os.getenv("OPENAI_API_KEY")

os.environ["GOOGLE_API_KEY"] = google_api_key.strip('"')
os.environ["NVIDIA_API_KEY"] = nvidia_api_key.strip('"')
os.environ["OPENAI_API_KEY"] = open_api_key.strip('"')
seed = 42

In [3]:
safety_settings = [
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
]

In [4]:
llm_config = {
    "config_list": [
        {
            "model": "gemini-1.5-flash",
            "api_key": os.environ["GOOGLE_API_KEY"],
            "api_type": "google",
            "safety_settings": safety_settings,
        }
    ]
}

In [6]:
coding_task = [
    """download data from https://raw.githubusercontent.com/vega/vega-datasets/main/data/penguins.json""",
    """ find desccriptive statistics of the dataset, plot a chart of their relation between spices and beak length and save the plot to beak_length_depth.png """,
    """Develope a short report using the data from the dataset, save it to a file named penguin_report.md.""",
]

In [7]:
user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "groupchat",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    human_input_mode="NEVER",
)
coder = autogen.AssistantAgent(
    name="Coder",  # the default assistant agent is capable of solving problems with code
    llm_config=llm_config,
)
writer = autogen.AssistantAgent(
    name="writer",
    llm_config=llm_config,
    system_message="""
        You are a professional report writer, known for
        your insightful and engaging report for clients.
        You transform complex concepts into compelling narratives.
        Reply "TERMINATE" in the end when everything is done.
        """,
)
critic = autogen.AssistantAgent(
    name="Critic",
    system_message="""Critic. You are a helpful assistant highly skilled in evaluating the quality of a given visualization code by providing a score from 1 (bad) - 10 (good) while providing clear rationale. YOU MUST CONSIDER VISUALIZATION BEST PRACTICES for each evaluation. Specifically, you can carefully evaluate the code across the following dimensions
- bugs (bugs):  are there bugs, logic errors, syntax error or typos? Are there any reasons why the code may fail to compile? How should it be fixed? If ANY bug exists, the bug score MUST be less than 5.
- Data transformation (transformation): Is the data transformed appropriately for the visualization type? E.g., is the dataset appropriated filtered, aggregated, or grouped  if needed? If a date field is used, is the date field first converted to a date object etc?
- Goal compliance (compliance): how well the code meets the specified visualization goals?
- Visualization type (type): CONSIDERING BEST PRACTICES, is the visualization type appropriate for the data and intent? Is there a visualization type that would be more effective in conveying insights? If a different visualization type is more appropriate, the score MUST BE LESS THAN 5.
- Data encoding (encoding): Is the data encoded appropriately for the visualization type?
- aesthetics (aesthetics): Are the aesthetics of the visualization appropriate for the visualization type and the data?

YOU MUST PROVIDE A SCORE for each of the above dimensions.
{bugs: 0, transformation: 0, compliance: 0, type: 0, encoding: 0, aesthetics: 0}
Do not suggest code.
Finally, based on the critique above, suggest a concrete list of actions that the coder should take to improve the code.
""",
    llm_config=llm_config,
)

groupchat_coder = autogen.GroupChat(
    agents=[user_proxy, coder, critic], messages=[], max_round=10
)

groupchat_writer = autogen.GroupChat(
    agents=[user_proxy, writer, critic], messages=[], max_round=10
)
manager_1 = autogen.GroupChatManager(
    groupchat=groupchat_coder,
    llm_config=llm_config,
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "groupchat",
        "use_docker": False,
    },
)

manager_2 = autogen.GroupChatManager(
    groupchat=groupchat_writer,
    name="Writing_manager",
    llm_config=llm_config,
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "groupchat",
        "use_docker": False,
    },
)

In [8]:
user = autogen.UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "tasks",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

In [9]:
user.initiate_chats(
    [
        {"recipient": coder, "message": coding_task[0], "summary_method": "last_msg"},
        {
            "recipient": manager_1,
            "message": coding_task[1],
            "summary_method": "last_msg",
        },
        {"recipient": manager_2, "message": coding_task[2]},
    ]
)


********************************************************************************
Starting a new chat....

********************************************************************************
User (to Coder):

download data from https://raw.githubusercontent.com/vega/vega-datasets/main/data/penguins.json

--------------------------------------------------------------------------------
Coder (to User):

```python
# filename: download_penguins.py
import requests

url = "https://raw.githubusercontent.com/vega/vega-datasets/main/data/penguins.json"
response = requests.get(url)

with open("penguins.json", "wb") as f:
    f.write(response.content)
```

Please execute the above python script. 
TERMINATE


--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
User (to chat_manager):

 f

[ChatResult(chat_id=None, chat_history=[{'content': 'download data from https://raw.githubusercontent.com/vega/vega-datasets/main/data/penguins.json', 'role': 'assistant'}, {'content': '```python\n# filename: download_penguins.py\nimport requests\n\nurl = "https://raw.githubusercontent.com/vega/vega-datasets/main/data/penguins.json"\nresponse = requests.get(url)\n\nwith open("penguins.json", "wb") as f:\n    f.write(response.content)\n```\n\nPlease execute the above python script. \nTERMINATE\n', 'role': 'user'}], summary='```python\n# filename: download_penguins.py\nimport requests\n\nurl = "https://raw.githubusercontent.com/vega/vega-datasets/main/data/penguins.json"\nresponse = requests.get(url)\n\nwith open("penguins.json", "wb") as f:\n    f.write(response.content)\n```\n\nPlease execute the above python script. \n\n', cost={'usage_including_cached_inference': {'total_cost': 0.005418, 'gemini-1.5-flash': {'cost': 0.005418, 'prompt_tokens': 498, 'completion_tokens': 92, 'total_toke

In [ ]:
# Example Teacher Student Agents

from autogen import ConversableAgent

# Setting up the agents.
student_agent = ConversableAgent(
    name="Student_Agent",
    system_message="You are a student willing to learn.",
    llm_config=llm_config,
)

teacher_agent = ConversableAgent(
    name="Teacher_Agent",
    system_message="You are a computer science teacher.",
    llm_config=llm_config,
)

# Initiate chat
chat_result = student_agent.initiate_chat(
    teacher_agent,
    message="How does deep learning relate to artificial intelligence?",
    summary_method="last_msg",
    max_turns=1, 
)